In [38]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

all_data = pd.read_csv('all_data.csv')

Preprocessing

In [39]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


# Handle NaN values
all_data.fillna(0, inplace=True)

# Convert columns to string type to ensure uniformity
for column in ['team_name', 'season', 'was_home']:
    all_data[column] = all_data[column].astype(str)

# Label encode 'team_name', 'season', and 'was_home'
label_encoders = {}  # To store label encoder objects for inverse transform
for column in ['team_name', 'season', 'was_home']:
    le = LabelEncoder()
    all_data[column] = le.fit_transform(all_data[column])
    label_encoders[column] = le

    # Save the mapping to CSV
    mapping = pd.DataFrame({
        'original': le.classes_,
        'encoded': range(len(le.classes_))
    })
    mapping.to_csv(f'{column}_mapping.csv', index=False)
# Dropping the 'name' column
all_data = all_data.drop(['name'], axis=1)


Columns to drop on RFE

In [40]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Define your X and y
X = all_data.drop(['total_points'], axis=1)
y = all_data['total_points']

# Perform RFE
model = LinearRegression()
rfe = RFE(model, n_features_to_select=40)
fit = rfe.fit(X, y)

# Get the features selected by RFE
selected_features = X.columns[fit.support_]
print("Selected Features:", selected_features)


Selected Features: Index(['was_home', 'season', 'team', 'team_name', 'ict_index_rolling_5',
       'ict_index_rolling_10', 'minutes_rolling_5', 'clean_sheets_rolling_5',
       'clean_sheets_rolling_10', 'value_rolling_5', 'influence_rolling_5',
       'influence_rolling_10', 'goals_conceded_rolling_5',
       'goals_conceded_rolling_10', 'threat_rolling_5', 'threat_rolling_10',
       'bps_rolling_5', 'bps_rolling_10', 'creativity_rolling_5',
       'creativity_rolling_10', 'lagged_value', 'lagged_ict_index',
       'lagged_minutes', 'lagged_ict_index_rolling_5',
       'lagged_ict_index_rolling_10', 'lagged_minutes_rolling_5',
       'lagged_clean_sheets_rolling_5', 'lagged_clean_sheets_rolling_10',
       'lagged_value_rolling_5', 'lagged_influence_rolling_5',
       'lagged_influence_rolling_10', 'lagged_goals_conceded_rolling_5',
       'lagged_goals_conceded_rolling_10', 'lagged_threat_rolling_5',
       'lagged_threat_rolling_10', 'lagged_bps_rolling_5',
       'lagged_bps_rolli

Columns to drop on L1

In [53]:
from sklearn.linear_model import LassoCV

# Using smaller eps and larger n_alphas
lasso = LassoCV(cv=5, eps=1e-9, n_alphas=100).fit(X, y)
lasso_coef = lasso.coef_

# Features with non-zero coefficients
lasso_selected_features = X.columns[lasso_coef != 0]
print("Features selected by Lasso:", lasso_selected_features)

# Limiting to 40 features if more were selected
if len(lasso_selected_features) > 40:
    lasso_selected_features = lasso_selected_features[:40]
print("Top 40 features selected by Lasso:", lasso_selected_features)
print(len(lasso_selected_features))

/Users/takoda/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 252293.8688245449, tolerance: 82.28038821926039
  model = cd_fast.enet_coordinate_descent_gram(
/Users/takoda/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 254988.1469112588, tolerance: 82.28038821926039
  model = cd_fast.enet_coordinate_descent_gram(
/Users/takoda/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 258452.11034269945, tolerance: 82.28038821926039
  model = cd_fast.enet_coordinate_descent_gram(
/Users/takoda/opt/anaconda3/lib/python3.9/site-package

Features selected by Lasso: Index(['element', 'was_home', 'season', 'GW', 'team_name', 'minutes_rolling_5',
       'value_rolling_5', 'transfers_in_rolling_5', 'transfers_in_rolling_10',
       'influence_rolling_10', 'bps_rolling_5', 'selected_rolling_5',
       'lagged_value', 'lagged_selected', 'lagged_influence', 'lagged_minutes',
       'lagged_minutes_rolling_5', 'lagged_minutes_rolling_10',
       'lagged_value_rolling_5', 'lagged_transfers_in_rolling_5',
       'lagged_transfers_in_rolling_10', 'lagged_influence_rolling_5',
       'lagged_threat_rolling_5', 'lagged_threat_rolling_10',
       'lagged_bps_rolling_5', 'lagged_creativity_rolling_5',
       'lagged_creativity_rolling_10', 'lagged_selected_rolling_5',
       'lagged_total_points_rolling_10', 'lagged_index_x_lagged_minutes',
       'lagged_value_x_lagged_influence', 'lagged_value_x_lagged_total_points',
       'lagged_value_x_lagged_ict_index', 'lagged_value_x_lagged_bps',
       'lagged_value_x_lagged_minutes', 'lagg

/Users/takoda/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.883e+05, tolerance: 1.034e+02
  model = cd_fast.enet_coordinate_descent(


Columns to drop on tree base

In [42]:
from sklearn.ensemble import RandomForestRegressor

# Fit the random forest model
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X, y)

# Get feature importances
importances = rf.feature_importances_

# Get the indices of the important features
indices = np.argsort(importances)[::-1]

# Select top 15 features based on their importances
rf_selected_features = X.columns[indices[:40]]
print("Features selected by Random Forest:", rf_selected_features)


Features selected by Random Forest: Index(['lagged_selected_x_lagged_minutes', 'transfers_in_rolling_5',
       'minutes_rolling_5', 'lagged_transfers_in_rolling_5', 'element', 'GW',
       'lagged_total_points_rolling_5', 'transfers_in_rolling_10',
       'lagged_selected', 'lagged_total_points_rolling_10',
       'selected_rolling_5', 'lagged_index_x_lagged_minutes',
       'lagged_selected_rolling_5', 'lagged_transfers_in_rolling_10',
       'ict_index_rolling_5', 'lagged_value_x_lagged_minutes',
       'creativity_rolling_5', 'influence_rolling_5', 'value_rolling_5',
       'bps_rolling_5', 'creativity_rolling_10', 'lagged_value',
       'lagged_influence_rolling_5', 'influence_rolling_10',
       'lagged_creativity_rolling_5', 'lagged_value_rolling_5',
       'lagged_creativity_rolling_10', 'team_name', 'lagged_minutes_rolling_5',
       'bps_rolling_10', 'lagged_value_x_lagged_total_points',
       'lagged_influence_rolling_10', 'minutes_rolling_10',
       'lagged_minutes_rollin

In [54]:
# Creating sets for each feature list
rfe_features = set(['was_home', 'season', 'team', 'team_name', 'ict_index_rolling_5',
       'ict_index_rolling_10', 'minutes_rolling_5', 'clean_sheets_rolling_5',
       'clean_sheets_rolling_10', 'value_rolling_5', 'influence_rolling_5',
       'influence_rolling_10', 'goals_conceded_rolling_5',
       'goals_conceded_rolling_10', 'threat_rolling_5', 'threat_rolling_10',
       'bps_rolling_5', 'bps_rolling_10', 'creativity_rolling_5',
       'creativity_rolling_10', 'lagged_value', 'lagged_ict_index',
       'lagged_minutes', 'lagged_ict_index_rolling_5',
       'lagged_ict_index_rolling_10', 'lagged_minutes_rolling_5',
       'lagged_clean_sheets_rolling_5', 'lagged_clean_sheets_rolling_10',
       'lagged_value_rolling_5', 'lagged_influence_rolling_5',
       'lagged_influence_rolling_10', 'lagged_goals_conceded_rolling_5',
       'lagged_goals_conceded_rolling_10', 'lagged_threat_rolling_5',
       'lagged_threat_rolling_10', 'lagged_bps_rolling_5',
       'lagged_bps_rolling_10', 'lagged_creativity_rolling_10',
       'lagged_total_points_rolling_5', 'lagged_total_points_rolling_10'])

lasso_features = set(['element', 'minutes_rolling_5', 'minutes_rolling_10',
       'transfers_in_rolling_5', 'transfers_in_rolling_10',
       'selected_rolling_5', 'lagged_selected', 'lagged_minutes_rolling_5',
       'lagged_transfers_in_rolling_5', 'lagged_transfers_in_rolling_10',
       'lagged_selected_rolling_5', 'lagged_index_x_lagged_minutes',
       'lagged_value_x_lagged_minutes', 'lagged_selected_x_lagged_minutes',
       'lagged_minutes_squared', 'lagged_minutes_cubed'])

rf_features = set(['lagged_selected_x_lagged_minutes', 'transfers_in_rolling_5',
       'minutes_rolling_5', 'lagged_transfers_in_rolling_5', 'element', 'GW',
       'lagged_total_points_rolling_5', 'transfers_in_rolling_10',
       'lagged_selected', 'lagged_total_points_rolling_10',
       'selected_rolling_5', 'lagged_index_x_lagged_minutes',
       'lagged_selected_rolling_5', 'lagged_transfers_in_rolling_10',
       'ict_index_rolling_5', 'lagged_value_x_lagged_minutes',
       'creativity_rolling_5', 'influence_rolling_5', 'value_rolling_5',
       'bps_rolling_5', 'creativity_rolling_10', 'lagged_value',
       'lagged_influence_rolling_5', 'influence_rolling_10',
       'lagged_creativity_rolling_5', 'lagged_value_rolling_5',
       'lagged_creativity_rolling_10', 'team_name', 'lagged_minutes_rolling_5',
       'bps_rolling_10', 'lagged_value_x_lagged_total_points',
       'lagged_influence_rolling_10', 'minutes_rolling_10',
       'lagged_minutes_rolling_10', 'lagged_bps_rolling_5',
       'lagged_ict_index_rolling_5', 'ict_index_rolling_10',
       'lagged_bps_rolling_10', 'lagged_ict_index_rolling_10',
       'threat_rolling_5'])

# Finding common features
common_features = rfe_features.intersection(rf_features)
print(f'Common features among all three methods: {common_features}')


Common features among all three methods: {'minutes_rolling_5', 'ict_index_rolling_10', 'lagged_bps_rolling_5', 'creativity_rolling_5', 'bps_rolling_5', 'lagged_influence_rolling_5', 'value_rolling_5', 'bps_rolling_10', 'creativity_rolling_10', 'lagged_influence_rolling_10', 'lagged_total_points_rolling_10', 'lagged_creativity_rolling_10', 'lagged_ict_index_rolling_5', 'lagged_ict_index_rolling_10', 'lagged_total_points_rolling_5', 'lagged_value', 'influence_rolling_5', 'ict_index_rolling_5', 'threat_rolling_5', 'team_name', 'lagged_bps_rolling_10', 'lagged_minutes_rolling_5', 'lagged_value_rolling_5', 'influence_rolling_10'}
